In [1]:
import pandas as pd
import random

In [2]:
dataset_file_path = 'GDM-TOPSIS.csv'
df = pd.read_csv(dataset_file_path)
df

,Alternative,Content Relevance,File Structure Compatibility,Data Representation Consistency,Execution Environment Compatibility,Dependency Version Compatibility
0,IDFlakies + iFixFlakies,4,4,4,4,3
1,IDFlakies + ODRepair,3,3,3,4,4
2,NonDex + iFixFlakies,2,1,1,4,3
3,NonDex + ODRepair,3,2,1,3,4
4,Shaker + iFixFlakies,2,1,1,4,3
5,Shaker + ODRepair,3,1,2,4,3
6,PolDet (JPF) + iFixFlakies,1,1,1,3,4
7,PolDet (JPF) + ODRepair,1,1,2,3,4
8,DTDetector + iFixFlakies,3,2,2,3,4
9,DTDetector + ODRepair,3,3,3,3,4


In [37]:
def distance_indicator(criterium_alt1, criterium_alt2):
    if (criterium_alt1 > criterium_alt2) :
        return 1
    elif (criterium_alt1 == criterium_alt2):
        return 0
    else:
        return -1
    

Define weights

In [38]:
weight_content_relevance = 0.4
weight_file_structure = 0.1
weight_data_representation = 0.1
weight_execution_environment = 0.2
weight_dependency_version = 0.2

weights = [weight_content_relevance,weight_file_structure,weight_data_representation,weight_execution_environment,weight_dependency_version]

ideal_positive= [4,4,4,4,4]
ideal_negative = [1,1,1,3,3]

Calculation (ideal positive)

In [40]:
#y - ideal positive
#z - current alternative
#j - other alternative
#k - current criterium
positives = []

#Overall calc
for idx, zRow in df.iterrows():
    z = zRow.drop('Alternative').values
    alternative = zRow['Alternative']
    

    #Numerator
    
    #Numerator First term
    numFirstTerm = 0
    for k in range(4):
      numFirstTerm = numFirstTerm + (weights[k] * distance_indicator(ideal_positive[k], z[k]) * distance_indicator(z[k],ideal_positive[k]))
        
    #Numerator Second term
    numSecondTerm = 0
    dfWithoutZ = df.drop(idx)
    for k in range(5):
        for idx,jRow in dfWithoutZ.drop('Alternative',axis=1).iterrows():
            j = jRow.values
            numSecondTerm = numSecondTerm + (weights[k] * distance_indicator(ideal_positive[k], j[k]) * distance_indicator(z[k], j[k]))

    numerator = numFirstTerm + numSecondTerm

    #Denominator

    #Denominator First term
    denFirstTerm = 0
    for k in range(5):
        for idx,jRow in dfWithoutZ.drop('Alternative',axis=1).iterrows():
            j = jRow.values
            denFirstTerm = denFirstTerm + (weights[k] * (distance_indicator(ideal_positive[k], j[k])**2))


    #Denominator Second term
    denSecondTerm = 0
    for k in range(5):
        for idx,jRow in dfWithoutZ.drop('Alternative',axis=1).iterrows():
            j = jRow.values
            denSecondTerm = denSecondTerm + (weights[k] * (distance_indicator(z[k], j[k])**2))

    denominator = 2*((denFirstTerm * denSecondTerm)**0.5)

    total = 0.5 - numerator/denominator

    positives.append((alternative, total))

result_df = pd.DataFrame(positives, columns=['Alternative', 'Positive Separation'])

Calculation (ideal negative)

In [41]:
negatives = []

#Overall calc
for idx, zRow in df.iterrows():
    z = zRow.drop('Alternative').values
    alternative = zRow['Alternative']
    

    #Numerator
    
    #Numerator First term
    numFirstTerm = 0
    for k in range(4):
      numFirstTerm = numFirstTerm + (weights[k] * distance_indicator(ideal_negative[k], z[k]) * distance_indicator(z[k],ideal_negative[k]))
        
    #Numerator Second term
    numSecondTerm = 0
    dfWithoutZ = df.drop(idx)
    for k in range(5):
        for idx,jRow in dfWithoutZ.drop('Alternative',axis=1).iterrows():
            j = jRow.values
            numSecondTerm = numSecondTerm + (weights[k] * distance_indicator(ideal_negative[k], j[k]) * distance_indicator(z[k], j[k]))

    numerator = numFirstTerm + numSecondTerm

    #Denominator

    #Denominator First term
    denFirstTerm = 0
    for k in range(5):
        for idx,jRow in dfWithoutZ.drop('Alternative',axis=1).iterrows():
            j = jRow.values
            denFirstTerm = denFirstTerm + (weights[k] * (distance_indicator(ideal_negative[k], j[k])**2))


    #Denominator Second term
    denSecondTerm = 0
    for k in range(5):
        for idx,jRow in dfWithoutZ.drop('Alternative',axis=1).iterrows():
            j = jRow.values
            denSecondTerm = denSecondTerm + (weights[k] * (distance_indicator(z[k], j[k])**2))

    denominator = 2*((denFirstTerm * denSecondTerm)**0.5)

    total = 0.5 - numerator/denominator

    negatives.append(total)

In [44]:
result_df['Negative Separation'] = negatives
result_df

,Alternative,Positive Separation,Negative Separation
0,IDFlakies + iFixFlakies,0.061273,0.750692
1,IDFlakies + ODRepair,0.148237,0.637620
2,NonDex + iFixFlakies,0.631787,0.168967
3,NonDex + ODRepair,0.378659,0.408955
4,Shaker + iFixFlakies,0.631787,0.168967
5,Shaker + ODRepair,0.346907,0.419592
6,PolDet (JPF) + iFixFlakies,0.796085,0.072525
7,PolDet (JPF) + ODRepair,0.738125,0.104839
8,DTDetector + iFixFlakies,0.319714,0.445418
9,DTDetector + ODRepair,0.244845,0.527037


Calculate Relative Closeness to Ideal Solution

In [46]:
closenesses = []

for idx, row in result_df.iterrows():
    closeness = row['Negative Separation']/(row['Positive Separation'] + row['Negative Separation'])
    closenesses.append(closeness)

result_df['Relative Closeness'] = closenesses
result_df_sorted = result_df.sort_values(by='Relative Closeness', ascending=False)
result_df_sorted

,Alternative,Positive Separation,Negative Separation,Relative Closeness
0,IDFlakies + iFixFlakies,0.061273,0.750692,0.924537
1,IDFlakies + ODRepair,0.148237,0.637620,0.811369
9,DTDetector + ODRepair,0.244845,0.527037,0.682795
8,DTDetector + iFixFlakies,0.319714,0.445418,0.582145
5,Shaker + ODRepair,0.346907,0.419592,0.547413
3,NonDex + ODRepair,0.378659,0.408955,0.519233
2,NonDex + iFixFlakies,0.631787,0.168967,0.211010
4,Shaker + iFixFlakies,0.631787,0.168967,0.211010
7,PolDet (JPF) + ODRepair,0.738125,0.104839,0.124369
6,PolDet (JPF) + iFixFlakies,0.796085,0.072525,0.083495


Calculate average closeness to hipothetical perfect solution (of value 1)	

In [55]:
#Overall average
average = result_df['Relative Closeness'].mean()
print("Overall Average: " ,average)

#OdRepair combinations average
averageOdRepair = result_df[result_df['Alternative'].str.contains('ODRepair')]['Relative Closeness'].mean()
print("ODRepair Average: " ,averageOdRepair)

#iFixFlakies combinations average
averageIFixFlakies= result_df[result_df['Alternative'].str.contains('iFixFlakies')]['Relative Closeness'].mean()
print("iFixFlakies Average: " ,averageIFixFlakies)

#iFixFlakies combinations average, excluding the outlier combination made by the same team
averageIFixFlakiesOutlier = result_df[~result_df['Alternative'].str.contains('IDFlakies') & result_df['Alternative'].str.contains('iFixFlakies')]['Relative Closeness'].mean()
print("iFixFlakies Average (excluding outlier): ",averageIFixFlakiesOutlier)


Overall Average:  0.4697377537223077
ODRepair Average:  0.5370359244281564
iFixFlakies Average:  0.40243958301645905
iFixFlakies Average (excluding outlier):  0.271915199562051
